# Movie Recommendation

### Imports and Data Sets

In [268]:
import numpy as np
import pandas as pd
import ast
import nltk
import re
import matplotlib.pyplot as plt
import requests

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from nltk.stem.porter import PorterStemmer
from tabulate import tabulate
from IPython.display import display, HTML, Image

import nltk
nltk.download('stopwords')

TMDB_API_KEY = 'a1fe2f0ac92d2dd849674115d68777a5'
# IMDB_API_KEY = '4ab406f9'

ps = PorterStemmer()
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
scaler = MinMaxScaler()

credits_df = pd.read_csv('credits.csv')
movies_df = pd.read_csv('movies.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\grans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Combine movies and credits df using title as key

In [270]:
movies_df = movies_df.merge(credits_df, on='title')
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'popularity']]

In [271]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4808 entries, 0 to 4807
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movie_id    4808 non-null   int64  
 1   title       4808 non-null   object 
 2   overview    4805 non-null   object 
 3   genres      4808 non-null   object 
 4   keywords    4808 non-null   object 
 5   cast        4808 non-null   object 
 6   crew        4808 non-null   object 
 7   popularity  4808 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 338.1+ KB


### Checking for and removing any empty or duplicated rows 

In [272]:
movies_df.isnull().sum()

movie_id      0
title         0
overview      3
genres        0
keywords      0
cast          0
crew          0
popularity    0
dtype: int64

In [273]:
movies_df.dropna(inplace=True)

In [274]:
movies_df.duplicated().sum()

0

In [275]:
movies_call_df = movies_df.copy()

### Making Genres and Keywords into a readable list

In [276]:
def convert_list(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

### Fetching the top 3 cast

In [277]:
def fetch_lead_actors(obj):
    L=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

### Fetching the Director(s)

In [278]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
    return L

### Stemming The Overview

In [279]:
port_stem = PorterStemmer()

def stemming(content):
    # Only keeping alphabets in the content 
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    # Removing spaces and converting to lowercase
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    # Stemming the words that are not in stopwords
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Joining the list of words into a string
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

### Using the above functions

In [280]:
movies_df['overview'] = movies_df['overview'].apply(stemming)
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

In [281]:
movies_df['genres'] = movies_df['genres'].apply(convert_list)

In [282]:
movies_df['keywords'] = movies_df['keywords'].apply(convert_list)

In [283]:
movies_df['cast'] = movies_df['cast'].apply(fetch_lead_actors)

In [284]:
movies_df['crew'] = movies_df['crew'].apply(fetch_director)

In [285]:
# If some popularity values might be 0, add 1 to avoid log(0)
movies_df['popularity_log'] = np.log1p(movies_df['popularity'])

movies_df['popularity_log_norm'] = scaler.fit_transform(movies_df[['popularity_log']])

### Removes all white space for comparison

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x:[i.replace(" ","") for i in x])

### New column with combined, overview, genres, keywords, cast and crew

In [ ]:
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

### New dataframe with 3 headers

In [ ]:
movies = movies_df[['movie_id', 'title', 'tags']]

###### Turns into a string rather than a list

In [ ]:
movies['tags'] = movies['tags'].apply(lambda x:' '.join(x))

C:\Users\grans\AppData\Local\Temp\ipykernel_26696\2816021526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x:' '.join(x))


In [ ]:
movies.head()

,movie_id,title,tags
0,19995,Avatar,nd centuri parapleg marin dispatch moon pandor...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believ dead come back li...
2,206647,Spectre,cryptic messag bond past send trail uncov sini...
3,49026,The Dark Knight Rises,follow death district attorney harvey dent bat...
4,49529,John Carter,john carter war weari former militari captain ...


In [ ]:
movies['tags'] = movies['tags'].apply(lambda x:x.lower())

C:\Users\grans\AppData\Local\Temp\ipykernel_26696\1170348455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x:x.lower())


In [ ]:
vectors = cv.fit_transform(movies['tags']).toarray()
len(cv.get_feature_names_out())

5000

### Uses ps (automatic NLP)

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
movies['tags'] = movies['tags'].apply(stem)

C:\Users\grans\AppData\Local\Temp\ipykernel_26696\3456099318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(stem)


In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def split_names(name):
    return re.sub(r'([a-z])([A-Z])', r'\1 \2', name)

In [291]:
def fetch_movie_details(movie_id):
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}&language=en-US'
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching details for movie ID {movie_id}: {e}")
        return None

In [294]:
def get_imdb_rating(movie_id):
    data = fetch_movie_details(movie_id)
    if not data or 'imdb_id' not in data:
        return None
    
    imdb_id = data['imdb_id']
    imdb_url = f"https://www.omdbapi.com/?i={imdb_id}&apikey={IMDB_API_KEY}"
    try:
        imdb_response = requests.get(imdb_url)
        imdb_response.raise_for_status()
        imdb_data = imdb_response.json()
        
        # The IMDb rating is typically in 'imdbRating' field
        if 'imdbRating' in imdb_data and imdb_data['imdbRating'] != 'N/A':
            return float(imdb_data['imdbRating'])  # Convert to float if you want
    except requests.exceptions.RequestException as e:
        print(f"Error fetching IMDb rating for IMDb ID {imdb_id}: {e}")

    return None

In [ ]:
def fetch_poster_path(movie_id):
    data = fetch_movie_details(movie_id)
    poster_path = data.get('poster_path')
    if poster_path:
        return f'https://image.tmdb.org/t/p/w500/{poster_path}'

In [ ]:
def get_poster_html(movie_id):
    poster_path = fetch_poster_path(movie_id)
    if poster_path:
        return f'<img src="{poster_path}" style="max-height:150px;">'
    else:
        return ''

In [ ]:
def get_movie_link_html(title):
    google_search_link = f'https://www.google.com/search?q={title}'
    return f'<a href="{google_search_link}" target="_blank">{title}</a>'

In [ ]:
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:26]
    
    # Create a dictionary to store the combined scores (similarity + popularity_weight * popularity)
    combined_scores = {}
    
    w_similarity = 0.9
    w_popularity = 0.1
    w_imbd = 0.1

    for i in movies_list:
        index = i[0]
        sim_score = i[1]
        
        pop_score = movies_df.iloc[index]['popularity_log_norm']

        # Calculate the combined score
        combined_score = (w_similarity * sim_score
                          + w_popularity * pop_score)

        combined_scores[index] = combined_score

    # Sort the dictionary by combined scores in descending order
    
    sorted_combined_scores = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    # Get the top 10 movies based on the combined scores
    movies_final_list = sorted_combined_scores[:10]
    
    return movies_final_list

    
    
def output_recommendations(movies_final_list):
    
    # Create an empty HTML string to accumulate the content
    html_content = ''

    # Add column headers
    headers_html = '<div style="display: flex; align-items: center; font-weight: bold; text-align: center; padding: 10px;">'
    headers_html += '<div style="flex: 1;">Number</div>'
    headers_html += '<div style="flex: 2;">Poster</div>'
    headers_html += '<div style="flex: 2;">Movie Title</div>'
    headers_html += '<div style="flex: 8;">Overview</div>'
    headers_html += '<div style="flex: 2;">Cast</div>'
    headers_html += '<div style="flex: 2;">Director</div>'
    headers_html += '</div>'
    html_content += headers_html

    for counter, i in enumerate(movies_final_list, start=1):
        title = movies_df.iloc[i[0]].title
        overview = movies_call_df.iloc[i[0]].overview
        cast = split_names('<br>'.join(movies_df.iloc[i[0]].cast))
        crew = split_names('<br>'.join(movies_df.iloc[i[0]].crew))
        movie_id = movies_df.iloc[i[0]].movie_id

        # Get HTML for the poster image
        poster_html = get_poster_html(movie_id)
        
        # Get clickable link HTML for the movie title
        movie_title_link_html = get_movie_link_html(title)

        # Build the row HTML
        row_html = '<div style="display: flex; align-items: center; text-align: center; padding: 10px;">'
        row_html += f'<div style="flex: 1;">{counter}</div>'
        row_html += f'<div style="flex: 2;">{poster_html}</div>'
        row_html += f'<div style="flex: 2;">{movie_title_link_html}</div>'
        row_html += f'<div style="flex: 8;">{overview}</div>'
        row_html += f'<div style="flex: 2;">{cast}</div>'
        row_html += f'<div style="flex: 2;">{crew}</div>'
        row_html += '</div>'

        # Append the row HTML to the content
        html_content += row_html

    # Display the accumulated HTML content
    display(HTML(html_content))

In [ ]:
movie = "Titanic"
recommendations = recommend(movie)
output_recommendations(recommendations)